In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf

### Read Company Return Data from `stkdata.sas7bdat`

In [5]:
stkdata = pd.read_sas('stkdata.sas7bdat', encoding='utf-8')

print(stkdata.head())

        DATE TICKER       RET
0 2011-01-31   AAPL  0.051959
1 2011-02-28   AAPL  0.040935
2 2011-03-31   AAPL -0.013314
3 2011-04-29   AAPL  0.004656
4 2011-05-31   AAPL -0.006569


In [6]:
print(stkdata.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 3 columns):
DATE      6000 non-null datetime64[ns]
TICKER    6000 non-null object
RET       6000 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 140.8+ KB
None


### Now we pick `CVX`, `JNJ`, and `PFE`

In [7]:
stkdata = stkdata[stkdata['TICKER'].isin(['CVX', 'JNJ', 'PFE'])]

In [8]:
print(stkdata.head())

           DATE TICKER       RET
1800 2011-01-31    CVX  0.040329
1801 2011-02-28    CVX  0.100495
1802 2011-03-31    CVX  0.036048
1803 2011-04-29    CVX  0.018141
1804 2011-05-31    CVX -0.034265


In [9]:
print(stkdata.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 1800 to 4559
Data columns (total 3 columns):
DATE      360 non-null datetime64[ns]
TICKER    360 non-null object
RET       360 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 11.2+ KB
None


### Read market data from `mktdata.sas7bdat`

In [10]:
mktdata = pd.read_sas('mktdata.sas7bdat', encoding='utf-8')

In [11]:
print(mktdata.head())

        DATE     SMB     HML   MKTRF      RF     UMD
0 2011-01-31 -0.0252  0.0082  0.0199  0.0001 -0.0029
1 2011-02-28  0.0153  0.0129  0.0349  0.0001  0.0208
2 2011-03-31  0.0258 -0.0176  0.0046  0.0001  0.0352
3 2011-04-29 -0.0037 -0.0243  0.0290  0.0000  0.0006
4 2011-05-31 -0.0058 -0.0205 -0.0127  0.0000 -0.0057


In [12]:
print(mktdata.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
DATE     120 non-null datetime64[ns]
SMB      120 non-null float64
HML      120 non-null float64
MKTRF    120 non-null float64
RF       120 non-null float64
UMD      120 non-null float64
dtypes: datetime64[ns](1), float64(5)
memory usage: 5.8 KB
None


### Obtain the summary statistics of variables in `mktdata`

Please note that the `10 percentile` of `MKTRF` is different from the answer of SAS. We have 120 observations, so the `10 percentile` is between the smallest 12th to the smallest 13th. SAS simply took the average of these two numbers, while Python here got slightly above the smallest 12th observation.

In [13]:
print(mktdata[['MKTRF','RF']].describe(percentiles=[.10,.90]))

            MKTRF          RF
count  120.000000  120.000000
mean     0.011495    0.000456
std      0.041239    0.000661
min     -0.133800    0.000000
10%     -0.033510    0.000000
50%      0.013250    0.000100
90%      0.055800    0.001600
max      0.136500    0.002100


### Merge these two data files: `stkdata` and `mktdata`

In [14]:
Regdata = pd.merge(stkdata, mktdata, on='DATE')

In [15]:
print(Regdata.head())

        DATE TICKER       RET     SMB     HML   MKTRF      RF     UMD
0 2011-01-31    CVX  0.040329 -0.0252  0.0082  0.0199  0.0001 -0.0029
1 2011-01-31    JNJ -0.033630 -0.0252  0.0082  0.0199  0.0001 -0.0029
2 2011-01-31    PFE  0.040548 -0.0252  0.0082  0.0199  0.0001 -0.0029
3 2011-02-28    CVX  0.100495  0.0153  0.0129  0.0349  0.0001  0.0208
4 2011-02-28    JNJ  0.036975  0.0153  0.0129  0.0349  0.0001  0.0208


In [16]:
print(Regdata.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 0 to 359
Data columns (total 8 columns):
DATE      360 non-null datetime64[ns]
TICKER    360 non-null object
RET       360 non-null float64
SMB       360 non-null float64
HML       360 non-null float64
MKTRF     360 non-null float64
RF        360 non-null float64
UMD       360 non-null float64
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 25.3+ KB
None


In [17]:
Regdata['RETRF'] = Regdata['RET'] - Regdata['RF']

In [18]:
print(Regdata)

          DATE TICKER       RET     SMB     HML   MKTRF      RF     UMD  \
0   2011-01-31    CVX  0.040329 -0.0252  0.0082  0.0199  0.0001 -0.0029   
1   2011-01-31    JNJ -0.033630 -0.0252  0.0082  0.0199  0.0001 -0.0029   
2   2011-01-31    PFE  0.040548 -0.0252  0.0082  0.0199  0.0001 -0.0029   
3   2011-02-28    CVX  0.100495  0.0153  0.0129  0.0349  0.0001  0.0208   
4   2011-02-28    JNJ  0.036975  0.0153  0.0129  0.0349  0.0001  0.0208   
..         ...    ...       ...     ...     ...     ...     ...     ...   
355 2020-11-30    JNJ  0.062577  0.0548  0.0211  0.1247  0.0001 -0.1225   
356 2020-11-30    PFE  0.140506  0.0548  0.0211  0.1247  0.0001 -0.1225   
357 2020-12-31    CVX -0.031315  0.0481 -0.0136  0.0463  0.0001 -0.0242   
358 2020-12-31    JNJ  0.087780  0.0481 -0.0136  0.0463  0.0001 -0.0242   
359 2020-12-31    PFE -0.039154  0.0481 -0.0136  0.0463  0.0001 -0.0242   

        RETRF  
0    0.040229  
1   -0.033730  
2    0.040448  
3    0.100395  
4    0.036875  
.. 

### Note:

We don't have missing observations in this case, so the default merge is good. Even if we have missing observations, we will skip them automatically when running regressions anyway.

### Summary statistics of `RETRF`

- We inclued `RETRF` and `TICKER` in the dataset for `.describe()`. `TICKER` is included because we use it in `.groupby()`.
- Again, the definition of `10 percentile` and `90 percentile` is slightly different from SAS.

In [19]:
print(Regdata[['RETRF','TICKER']].groupby('TICKER').describe(percentiles=[.10,.90]))

        RETRF                                                              \
        count      mean       std       min       10%       50%       90%   
TICKER                                                                      
CVX     120.0  0.004405  0.067699 -0.224898 -0.073128  0.006645  0.071186   
JNJ     120.0  0.010733  0.043058 -0.123411 -0.040997  0.012730  0.066523   
PFE     120.0  0.010566  0.053385 -0.143855 -0.046178  0.005749  0.075582   

                  
             max  
TICKER            
CVX     0.272850  
JNJ     0.144208  
PFE     0.188279  


### Run regression for each firm at once

- We extract the observations from the same company to `tempdf` with the `for` loop each time

In [20]:
for TIC in ['CVX', 'JNJ', 'PFE']:
    tempdf = Regdata[Regdata['TICKER'] == TIC]
    mdl = smf.ols('RETRF ~ MKTRF', data=tempdf).fit()
    print()
    print(TIC)
    print()
    print(mdl.summary())


CVX

                            OLS Regression Results                            
Dep. Variable:                  RETRF   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     133.5
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           4.11e-21
Time:                        22:07:13   Log-Likelihood:                 198.75
No. Observations:                 120   AIC:                            -393.5
Df Residuals:                     118   BIC:                            -387.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0093      0.004     -2.116   

### Multiple Regressions (Not Required in this homework)

Please note that we use `df.unique()` to generate a list of companies instead of a given list in this example.

In [21]:
for TIC in Regdata['TICKER'].unique():
    tempdf = Regdata[Regdata['TICKER'] == TIC]
    mdl = smf.ols('RETRF ~ MKTRF + SMB + HML + UMD', data=tempdf).fit()
    print()
    print(TIC)
    print()
    print(mdl.summary())


CVX

                            OLS Regression Results                            
Dep. Variable:                  RETRF   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.619
Method:                 Least Squares   F-statistic:                     49.31
Date:                Sat, 27 Mar 2021   Prob (F-statistic):           4.27e-24
Time:                        22:09:02   Log-Likelihood:                 213.28
No. Observations:                 120   AIC:                            -416.6
Df Residuals:                     115   BIC:                            -402.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0037      0.004     -0.912   